In [15]:
import pandas as pd

clean_df= pd.read_csv("../results/qatar2025_f1_data.csv")

In [16]:
columns = [
    "AvgQualiPos_2025",
    "AvgRacePos_2025",
    "AvgSprintQualiPos_2025",
    "AvgSprintPos_2025"
]

for col in columns:
    clean_df[col] = clean_df[col].fillna(clean_df[col].mean())

In [17]:
for col in columns:
    mean = clean_df[col].mean()
    std = clean_df[col].std()
    clean_df[col + "_z"] = (clean_df[col] - mean) / std

In [18]:
for col in columns:
    clean_df[col + "_z_inv"] = -clean_df[col + "_z"]

In [19]:
qatar_mean = clean_df["QatarLapTime"].mean()
qatar_std = clean_df["QatarLapTime"].std()

In [20]:
clean_df["QatarLapTime_z"] = (clean_df["QatarLapTime"] - qatar_mean) / qatar_std

In [21]:
clean_df["QatarPace_z_inv"] = -clean_df["QatarLapTime_z"]

In [22]:
w_quali = 1.0
w_race = 1.0
w_sprint_quali = 0.7
w_sprint = 0.7
w_qatar = 1.3 

In [23]:
clean_df["QatarPace_z_inv_filled"] = clean_df["QatarPace_z_inv"].fillna(0.0)

In [24]:
clean_df["PerformanceScore"] = (
    w_quali        * clean_df["AvgQualiPos_2025_z_inv"] +
    w_race         * clean_df["AvgRacePos_2025_z_inv"] +
    w_sprint_quali * clean_df["AvgSprintQualiPos_2025_z_inv"] +
    w_sprint       * clean_df["AvgSprintPos_2025_z_inv"] +
    w_qatar        * clean_df["QatarPace_z_inv_filled"]
)

In [29]:
results_df = clean_df[["Driver", "Team", "PerformanceScore"]].copy()

ranking = (
     results_df.sort_values("PerformanceScore", ascending=False)
      .reset_index(drop=True)
)

ranking.head(15)

,Driver,Team,PerformanceScore
0,Lando Norris,McLaren Mercedes,5.605777
1,Max Verstappen,Red Bull Racing Honda RBPT,5.434732
2,Oscar Piastri,McLaren Mercedes,5.021879
3,Lewis Hamilton,Ferrari,4.802707
4,George Russell,Mercedes,4.444869
5,Charles Leclerc,Ferrari,4.157323
6,Kimi Antonelli,Mercedes,2.127603
7,Fernando Alonso,Aston Martin Aramco Mercedes,0.270369
8,Isack Hadjar,Racing Bulls Honda RBPT,-0.111845
9,Yuki Tsunoda,Racing Bulls Honda RBPT,-0.117620


In [30]:
team_view = (
    ranking[["Team", "Driver", "PerformanceScore"]]
    .sort_values(["Team", "PerformanceScore"], ascending=[True, False])
)

team_view["TeamRank"] = team_view.groupby("Team")["PerformanceScore"].rank(ascending=False, method="first")

team_view

,Team,Driver,PerformanceScore,TeamRank
10,Alpine Renault,Pierre Gasly,-0.168770,1.0
15,Alpine Renault,Franco Colapinto,-2.526822,2.0
17,Alpine Renault,Jack Doohan,-4.115471,3.0
7,Aston Martin Aramco Mercedes,Fernando Alonso,0.270369,1.0
14,Aston Martin Aramco Mercedes,Lance Stroll,-2.499735,2.0
3,Ferrari,Lewis Hamilton,4.802707,1.0
5,Ferrari,Charles Leclerc,4.157323,2.0
13,Haas Ferrari,Esteban Ocon,-1.889426,1.0
16,Haas Ferrari,Oliver Bearman,-3.278816,2.0
18,Kick Sauber Ferrari,Gabriel Bortoleto,-4.326899,1.0


In [32]:
ranking.to_csv("../results/qatar2025_driver_predictions.csv", index=False)
team_view.to_csv("../results/qatar2025_team_view.csv", index=False)

In [33]:
clean_df.to_csv("../data/clean_df.csv", index=False)